In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/fffiiibbbeee/train.csv" ,  encoding = 'latin')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("/kaggle/input/fffiiibbbeee/train.csv", encoding='latin')

# Check the distribution of the target variable to confirm counts
category_counts = df['target'].value_counts()

# Find the second highest number of samples
second_highest_count = category_counts.sort_values(ascending=False).iloc[1]

# Downsample the 'academic interests' category
df_academic = df[df['target'] == 'academic interests']
df_academic_downsampled = df_academic.sample(n=second_highest_count, random_state=42)

# Filter out the old 'academic interests' entries from the original dataframe
df_rest = df[df['target'] != 'academic interests']

# Concatenate the downsampled 'academic interests' dataframe with the rest of the data
df_final = pd.concat([df_rest, df_academic_downsampled])

# Optionally, shuffle the dataset if necessary
df_final = df_final.sample(frac=1).reset_index(drop=True)

df = df_final

# Print the new count of 'academic interests' to confirm
plt.figure(figsize=(8,6))
sns.countplot(y='target', data=df)
plt.title('Target Variable Distribution')
plt.ylabel('Target')
plt.xlabel('Count')
plt.show()



In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_val, y_train, y_val = train_test_split(df["text"],df["target"] , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True) 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output = False)
train_labels_one_hot  = one_hot_encoder.fit_transform(y_train.to_numpy().reshape(-1,1))
val_labels_one_hot = one_hot_encoder.transform(y_val.to_numpy().reshape(-1,1))

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded  = label_encoder.fit_transform(y_train.to_numpy())
val_labels_encoded = label_encoder.transform(y_val.to_numpy())

In [ ]:
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_

num_classes , class_names

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
sent_lens = [len(sen.split()) for sen in X_train]
output_seq_len = int(np.percentile(sent_lens , 95))
max_tokens = 68000

In [ ]:
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = max_tokens,
    output_sequence_length = output_seq_len
)

In [ ]:
text_vectorizer.adapt(X_train)

In [ ]:
vocab = text_vectorizer.get_vocabulary()

In [ ]:
token_embed = layers.Embedding(input_dim = len(vocab),
                               output_dim = 128,
                               mask_zero = True,
                               name = "token_embedding")

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,train_labels_one_hot ))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_val,val_labels_one_hot ))

In [ ]:
train_dataset  = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
inputs = layers.Input(shape = (1,), dtype = tf.string)
text_vectors = text_vectorizer(inputs)
token_embeddings = token_embed(text_vectors)
x = layers.Conv1D(128, 5, activation='relu')(token_embeddings)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(128, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(26, activation='softmax')(x)

model_1 = tf.keras.Model(inputs , outputs)

model_1.compile(
    loss = "categorical_crossentropy",
    optimizer = "Adam",
    metrics =["accuracy"]
)

model_1.summary()

In [ ]:
history_model_1 = model_1.fit(
                     train_dataset,
                     steps_per_epoch = int(0.1*len(train_dataset)),
                     epochs = 10,
                     validation_data = valid_dataset,
                     validation_steps = int(0.1*len(valid_dataset))
)

In [ ]:
model_1_pred_probs = model_1.predict(valid_dataset)

In [ ]:
model_1_preds = tf.argmax(model_1_pred_probs , axis = 1)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(val_labels_encoded, model_1_preds , average='weighted'))

In [ ]:
test_data = pd.read_csv("/kaggle/input/fffiiibbbeee/test.csv" ,  encoding = 'latin')
df_test.head()

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


# Extract the 'text' column for prediction
test_texts = test_data['text'].tolist()

# Create a TensorFlow Dataset
test_dataset = tf.data.Dataset.from_tensor_slices(test_texts)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Make predictions
predictions = model_1.predict(test_dataset)

# Get the predicted class indices
predicted_class_indices = np.argmax(predictions, axis=1)

# Convert class indices back to original labels
predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

# Add the predicted labels to the test DataFrame
test_data['target'] = predicted_labels

# Save the DataFrame to a new CSV file
output_file = 'predictions.csv'
test_data.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")



In [ ]:
op = pd.read_csv("/kaggle/working/predictions.csv" ,  encoding = 'latin')
op.head()


In [ ]:
op = op.drop(['text', 'Word Count'], axis=1)
op.head()

In [ ]:
op = op[[ 'target','Index']]

In [ ]:
op.head()

In [ ]:
op.to_csv("final_test.csv", index=False)
